In [30]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

##import libs


In [38]:
#import csv files
raw = pd.read_csv('Customers.csv')
pro = pd.read_csv('Products.csv')
tran= pd.read_csv('Transactions.csv')

In [40]:
#mergeing data
transactions_with_pro = pd.merge(tran, pro, on='ProductID', how='left')
customer_data = pd.merge(transactions_with_pro, customers, on='CustomerID', how='left')

In [42]:
profile = customer_data.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    transaction_count=('TransactionID', 'count'),
    avg_transaction_value=('TotalValue', 'mean'),
    total_quantity=('Quantity', 'sum')
).reset_index()

In [44]:
category_frequency = customer_data.groupby(['CustomerID', 'Category']).agg(
    product_frequency=('ProductID', 'count')
).reset_index()

In [46]:
category_matrix = category_frequency.pivot_table(index='CustomerID', columns='Category', values='product_frequency', fill_value=0)

In [48]:
customer_features = pd.merge(customer_profile, category_matrix, on='CustomerID', how='left')

In [50]:

scaler = StandardScaler()
normalized_features = scaler.fit_transform(customer_features.drop('CustomerID', axis=1))

In [52]:
similarity_matrix = cosine_similarity(normalized_features)

In [54]:
lookalike_results = {}


for idx, customer_id in enumerate(customer_features['CustomerID'].iloc[:20]):
    similarity_scores = similarity_matrix[idx]
    
    # Exclude the customer itself (it will have the highest similarity score)
    sorted_indices = similarity_scores.argsort()[-4:-1] 
    
    # Store the lookalikes and their similarity scores
    lookalike_results[customer_id] = [(customer_features['CustomerID'].iloc[i], similarity_scores[i]) for i in sorted_indices]

# Convert the results to a DataFrame
lookalike_df = pd.DataFrame(
    [(customer_id, val[0][0], val[0][1], val[1][0], val[1][1], val[2][0], val[2][1]) for customer_id, val in lookalike_results.items()],
    columns=['CustomerID', 'Customer1', 'Score1', 'Customer2', 'Score2', 'Customer3', 'Score3']
)




In [ ]:
lookalike_df.to_csv('Lookalike.csv', index=False)